# Calculate perplexity of GPT and SFT1RL1 without keyword control


Calculate 50 texts with GPT and strl1RL1 without keyowrd control and starting with "Wir waren in dem Hotel". As in the notebook "Evaluate model longer" the STR!RL1 is used to generate texts based on sentiment and first tokens from original review.

In [1]:
!pip install transformers trl rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, pipeline,AutoModelWithLMHead
import math
import torch
import numpy as np
import random
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


## Top k

In [ ]:
sentiment_pipe_kwargs = {"top_k": 15,"max_length":100,"pad_token_id":50256}

gpt = pipeline("text-generation", model="benjamin/gpt2-wechsel-german",**sentiment_pipe_kwargs)
sft1rl1 = pipeline("text-generation", model="/content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft_1_epoch_rl_1epochs",**sentiment_pipe_kwargs)

Some weights of the model checkpoint at /content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft_1_epoch_rl_1epochs were not used when initializing GPT2LMHeadModel: ['v_head.summary.weight', 'v_head.summary.bias']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokenizer_for_perplex = AutoTokenizer.from_pretrained("distilbert-base-german-cased")
model_for_perplex = AutoModelWithLMHead.from_pretrained("distilbert-base-german-cased")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
perplexity_gpt=[]
for i in range(0,50):
  generated_text=gpt("Wir waren in diesem Hotel.")
  generated_text=generated_text[0]["generated_text"]
  tokenize_input = tokenizer_for_perplex.tokenize(generated_text)
  tensor_input = torch.tensor([tokenizer_for_perplex.convert_tokens_to_ids(tokenize_input)])
  loss, logits = model_for_perplex(tensor_input, labels=tensor_input)[:2]
  ppl=math.exp(loss)
  perplexity_gpt.append(ppl)

In [ ]:
np.mean(perplexity_gpt)

1.0917974092915588

## SFT1RL1

In [ ]:
tokens=["[positive] ","[negative] "]

perplexity_sft1rl1=[]
for i in range(0,50):
  token=random.sample(tokens,1)
  generated_text=sft1rl1(token[0]+"Wir waren in diesem Hotel.")
  generated_text=generated_text[0]["generated_text"]
  tokenize_input = tokenizer_for_perplex.tokenize(generated_text)
  tensor_input = torch.tensor([tokenizer_for_perplex.convert_tokens_to_ids(tokenize_input)])
  loss, logits = model_for_perplex(tensor_input, labels=tensor_input)[:2]
  ppl=math.exp(loss)
  perplexity_sft1rl1.append(ppl)


1.127202040441388

## Top p

In [3]:
sentiment_pipe_kwargs = {"top_p": 0.5,"max_length":100,"pad_token_id":50256}

gpt = pipeline("text-generation", model="benjamin/gpt2-wechsel-german",**sentiment_pipe_kwargs)
sft1rl1 = pipeline("text-generation", model="/content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft_1_epoch_rl_1epochs",**sentiment_pipe_kwargs)

config.json:   0%|          | 0.00/947 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/665M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/250 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/856k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/514k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Some weights of the model checkpoint at /content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft_1_epoch_rl_1epochs were not used when initializing GPT2LMHeadModel: ['v_head.summary.weight', 'v_head.summary.bias']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
tokenizer_for_perplex = AutoTokenizer.from_pretrained("distilbert-base-german-cased")
model_for_perplex = AutoModelWithLMHead.from_pretrained("distilbert-base-german-cased")

In [5]:
perplexity_gpt=[]
for i in range(0,50):
  generated_text=gpt("Wir waren in diesem Hotel.")
  generated_text=generated_text[0]["generated_text"]
  tokenize_input = tokenizer_for_perplex.tokenize(generated_text)
  tensor_input = torch.tensor([tokenizer_for_perplex.convert_tokens_to_ids(tokenize_input)])
  loss, logits = model_for_perplex(tensor_input, labels=tensor_input)[:2]
  ppl=math.exp(loss)
  perplexity_gpt.append(ppl)

In [6]:
np.mean(perplexity_gpt)

1.0702424471886052

## SFT1RL1

In [15]:
tokens=["[positive] ","[negative] "]

perplexity_sft1rl1=[]
for i in range(0,50):
  token=random.sample(tokens,1)
  generated_text=sft1rl1(token[0]+"Wir waren in diesem Hotel.")
  generated_text=generated_text[0]["generated_text"]
  tokenize_input = tokenizer_for_perplex.tokenize(generated_text)
  tensor_input = torch.tensor([tokenizer_for_perplex.convert_tokens_to_ids(tokenize_input)])
  loss, logits = model_for_perplex(tensor_input, labels=tensor_input)[:2]
  ppl=math.exp(loss)
  perplexity_sft1rl1.append(ppl)


In [16]:
np.mean(perplexity_sft1rl1)

1.138831909679962

## Gready Search

In [9]:
sentiment_pipe_kwargs = {"top_k": 1,"max_length":100,"pad_token_id":50256}

gpt = pipeline("text-generation", model="benjamin/gpt2-wechsel-german",**sentiment_pipe_kwargs)
sft1rl1 = pipeline("text-generation", model="/content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft_1_epoch_rl_1epochs",**sentiment_pipe_kwargs)

Some weights of the model checkpoint at /content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft_1_epoch_rl_1epochs were not used when initializing GPT2LMHeadModel: ['v_head.summary.weight', 'v_head.summary.bias']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokenizer_for_perplex = AutoTokenizer.from_pretrained("distilbert-base-german-cased")
model_for_perplex = AutoModelWithLMHead.from_pretrained("distilbert-base-german-cased")

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/240k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/479k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


model.safetensors:   0%|          | 0.00/270M [00:00<?, ?B/s]

In [10]:
perplexity_gpt=[]
for i in range(0,50):
  generated_text=gpt("Wir waren in diesem Hotel.")
  generated_text=generated_text[0]["generated_text"]
  tokenize_input = tokenizer_for_perplex.tokenize(generated_text)
  tensor_input = torch.tensor([tokenizer_for_perplex.convert_tokens_to_ids(tokenize_input)])
  loss, logits = model_for_perplex(tensor_input, labels=tensor_input)[:2]
  ppl=math.exp(loss)
  perplexity_gpt.append(ppl)

In [11]:
np.mean(perplexity_gpt)

1.0660369771376461

## SFT1RL1

In [12]:
tokens=["[positive] ","[negative] "]

perplexity_sft1rl1=[]
for i in range(0,50):
  token=random.sample(tokens,1)
  generated_text=sft1rl1(token[0]+"Wir waren in diesem Hotel.")
  generated_text=generated_text[0]["generated_text"]
  tokenize_input = tokenizer_for_perplex.tokenize(generated_text)
  tensor_input = torch.tensor([tokenizer_for_perplex.convert_tokens_to_ids(tokenize_input)])
  loss, logits = model_for_perplex(tensor_input, labels=tensor_input)[:2]
  ppl=math.exp(loss)
  perplexity_sft1rl1.append(ppl)


In [13]:
np.mean(perplexity_sft1rl1)

1.1263263589245824